# Generate towers
The purpose of this notebook is to generate a set of towers, evaluate them according to basic metrics and to save them to disk.

In [1]:
import tower_generator

import tqdm

import pickle

import numpy as np
import matplotlib.pyplot as plt

from utils.blockworld_library import bl_silhouette2_default,bl_nonoverlapping_simple
from utils.blockworld import *

from model.BFS_Lookahead_Agent import BFS_Lookahead_Agent
from model.BFS_Agent import BFS_Agent

In [2]:
towers = []

In [3]:
generator = tower_generator.TowerGenerator(8,8,
                                        block_library=bl_nonoverlapping_simple,
                                        seed=42,
                                        )

In [4]:
for i in tqdm.tqdm(range(100)):
    towers.append(generator.generate())

100%|██████████| 100/100 [00:00<00:00, 134.29it/s]


In [5]:
def get_tower_cost(agent,tower):
    cost = 0
    world = Blockworld(silhouette=tower['bitmap'],
                       block_library=tower['block_library'],
                       physics=True,
                       legal_action_space=True)
    agent.set_world(world)
    while world.status()[0] == 'Ongoing':
        _,step_info = agent.act(1)
        cost += step_info['states_evaluated']
    return cost,world.status()
    

In [6]:
agent = BFS_Agent(random_seed=42)

In [7]:
get_tower_cost(agent,towers[0])

(53653, ('Win', 'None'))

In [9]:
for tower in tqdm.tqdm(towers):
    cost,status = get_tower_cost(agent,tower)
    tower['cost'] = cost
    tower['success'] = status[0] == 'Win'

 47%|████▋     | 47/100 [19:43<05:25,  6.14s/it]

In [ ]:
# pickle the towers
pickle.dump(towers,open('generated_evaluated_towers.pkl','wb'))